# LVL2 Script

In [4]:
#import external libraries
import pandas as pd
import os
#pd.options.mode.chained_assignment = None  # default='warn'; chained index warning
import numpy as np
import imp
import datetime
import pytz

#plotting
%matplotlib notebook

#import self-written libraries
import LVL2

In [5]:
imp.reload(LVL2)

<module 'LVL2' from 'C:\\Users\\ehbaker\\Documents\\Python\\WXmunge\\LVL2.py'>

In [11]:
#Data Import Tasks
#path to level 1 data
Glacier="Wolverine" #or "Gulkana"
Station="990" #or other elevations; this depends on the naming convention of input data
timezone='America/Anchorage' #choose from pytz.all_timezones
pth=r"Q:/Project Data/GlacierData/Benchmark_Program/Data/" +Glacier+ r"/AllYears/Wx/LVL1/emily/" + Glacier.lower()+"_990_ALL_15minLVL1.csv"

#time format and column
Local_time_column_name='Local_time'
date_format='%Y/%m/%d %H:%M'

#directory to save output data
save_pth=r"Q:/Project Data/GlacierData/Benchmark_Program/Data/" +Glacier+ r"/AllYears/Wx/LVL2/emily/" + Glacier.lower()+"_990_ALL_15minLVL2.csv"

In [12]:
#read in level1 CSV
dat=pd.read_csv(pth)

In [32]:
dat['local_time']=pd.to_datetime(dat[Local_time_column_name], format= date_format)
dat=dat.set_index('local_time') #set this local time as the index

In [34]:
temp_columns=['Tpassive1', 'Tpassive2', 'TAspirated1', 'TAspirated2']
precip_columns=['StageIncremental', 'TPGIncremental']

In [58]:
daily_dat=pd.DataFrame() #create empty dataframe
for temp_col in temp_columns:
    #Create WMO Mean Temp
    daily_dat[temp_col+"_min"]=dat[temp_col].resample('D').min()
    daily_dat[temp_col+ "_max"]=dat[temp_col].resample('D').max()
    daily_dat[temp_col+"_WMO"]=daily_dat[[temp_col+"_min", temp_col+'_max']].mean(axis=1)
    #Create USGS Mean Temp
    daily_dat[temp_col+"_USGS"]=dat[temp_col].resample('D').mean()

In [60]:
#Tasp1 will be the main temperature used @ Wolv990 - others, used for gap-filling
#If gap <=3 msmts, aka under an hour, fill with mean of 2 surrounding
#If longer, fill with avg of passive sensors

In [59]:
daily_dat

,Tpassive1_min,Tpassive1_max,Tpassive1_WMO,Tpassive1_USGS,Tpassive2_min,Tpassive2_max,Tpassive2_WMO,Tpassive2_USGS,TAspirated1_min,TAspirated1_max,TAspirated1_WMO,TAspirated1_USGS,TAspirated2_min,TAspirated2_max,TAspirated2_WMO,TAspirated2_USGS
local_time,,,,,,,,,,,,,,,,
2010-07-01,3.6,6.4,5.00,4.565574,3.2,6.0,4.60,4.154098,3.50,6.50,5.000,4.562295,NaN,NaN,NaN,NaN
2010-07-02,2.7,6.4,4.55,4.129167,2.3,6.0,4.15,3.721875,2.80,6.20,4.500,4.100000,NaN,NaN,NaN,NaN
2010-07-03,3.2,5.5,4.35,4.087500,2.8,5.0,3.90,3.673958,3.30,5.40,4.350,4.119792,NaN,NaN,NaN,NaN
2010-07-04,4.3,9.2,6.75,5.908333,3.9,8.8,6.35,5.513542,4.20,9.00,6.600,5.740625,NaN,NaN,NaN,NaN
2010-07-05,3.2,7.4,5.30,4.739583,2.7,6.9,4.80,4.306250,3.10,7.20,5.150,4.736458,NaN,NaN,NaN,NaN
2010-07-06,3.4,6.8,5.10,4.563542,2.9,6.3,4.60,4.145833,3.40,6.90,5.150,4.567708,NaN,NaN,NaN,NaN
2010-07-07,3.8,6.4,5.10,5.110417,3.4,6.0,4.70,4.691667,3.80,6.70,5.250,5.141667,NaN,NaN,NaN,NaN
2010-07-08,3.9,7.3,5.60,5.835417,3.5,6.8,5.15,5.407292,4.00,7.50,5.750,5.865625,NaN,NaN,NaN,NaN
2010-07-09,2.2,12.8,7.50,6.796875,1.8,12.3,7.05,6.373958,2.10,12.50,7.300,6.785417,NaN,NaN,NaN,NaN
